In [1]:
import datetime
import time
import subprocess
import sys
import pandas as pd, numpy as np
import os; os.chdir('..')

In [2]:
threshold = .02
step_size = .01
iterations = 100

du_targets = {0 : 0.218251,
           1 : 0.221090,
           2 : 0.151014,
           3 : 0.084200,
           4 : 0.162740,
           5 : 0.141933,
           6 : 0.020772}

nrsf_targets = {0 : 0.068475,
               1 : 0.268959,
               2 : 0.209235,
               3 : 0.046141,
               4 : 0.222778,
               5 : 0.184189,
               6 : 0.000223}

In [3]:
for i in range(iterations):
    print 'ITERATION %s' % (i + 1)
    start_time = datetime.datetime.now()
    print 'Starting iteration at: %s' % start_time
    
    # Simulation run
    subprocess.check_call([sys.executable, 'c://sandag//sandag_urbansim//calib_simulation.py'])
    
    # Read MSA results
    time.sleep(1)
    proportion_du_by_msa = pd.read_csv('.\\data\\calibration\\msa_du_simulated.csv').set_index('msa_id').to_dict()['residential_units']
    proportion_nrsf_by_msa = pd.read_csv('.\\data\\calibration\\msa_nrsf_simulated.csv').set_index('msa_id').to_dict()['non_residential_sqft']
    
    # Get existing shifters
    calibration_shifters = pd.read_csv('.\\data\\calibration\\msa_shifters.csv').set_index('msa_id')
    new_calib_shifters = pd.DataFrame(index = calibration_shifters.index)
    calibration_shifters = calibration_shifters.to_dict()
    
    completed = 0

    # Compare simulated with target, and adjust MSA shifters as needed
    for shifter_name in ['res_price_shifter', 'nonres_price_shifter']:
        print '**** ' + shifter_name + ' ****' 
        new_shifters = {}
        differences = []
        num_matching = 0
        shifters = calibration_shifters[shifter_name]
        for msa_id in shifters.keys():
            shift = shifters[msa_id]
            print 'MSA %s where current shifter value is %s' % (msa_id, shift)
            
            if shifter_name == 'res_price_shifter':
                target = du_targets[msa_id]
                simulated = proportion_du_by_msa[msa_id] if msa_id in proportion_du_by_msa else 0.0
            elif shifter_name == 'nonres_price_shifter':
                target = nrsf_targets[msa_id]
                simulated = proportion_nrsf_by_msa[msa_id] if msa_id in proportion_nrsf_by_msa else 0.0
            print '    Target proportion is %s' % target
            print '    Simulated proportion is %s' % simulated
            
            difference = target - simulated
            differences.append(abs(difference))

            if abs(difference) > threshold:
                if difference > 0:
                    print '        Action: shift up'
                    new_shift = shift + step_size
                else:
                    print '        Action: shift down'
                    new_shift = shift - step_size
                print '        New shift value %s' % new_shift
            else:
                print '        No action'
                new_shift = shift
                num_matching += 1

            new_shifters[msa_id] = new_shift
        
        # If all geographies matching within threshold, exit calibration
        print 'Number of matching MSAs: %s' % num_matching
        print 'Mean difference: %s' % pd.Series(differences).mean()
        if num_matching >= (len(du_targets) - 3):
            print 'Calibration complete for %s ' % shifter_name
            new_shifters = shifters
            completed += 1
            if completed == 2:
                break

        new_shifters = pd.Series(new_shifters, name = shifter_name)
        new_shifters.index.name = 'msa_id'
        new_calib_shifters[shifter_name] = new_shifters
        
    if completed < 2:
        new_calib_shifters.to_csv('.\\data\\calibration\\msa_shifters.csv')
    else:
        break
    
    end_time = datetime.datetime.now()
    print 'Iteration elapsed time: %s' % (end_time - start_time)
    time.sleep(1)

ITERATION 1
Starting iteration at: 2015-05-28 08:23:30.012000
**** res_price_shifter ****
MSA 0 where current shifter value is 0.29
    Target proportion is 0.218251
    Simulated proportion is 0.250543028144
        Action: shift down
        New shift value 0.28
MSA 1 where current shifter value is 0.26
    Target proportion is 0.22109
    Simulated proportion is 0.278952385417
        Action: shift down
        New shift value 0.25
MSA 2 where current shifter value is 0.51
    Target proportion is 0.151014
    Simulated proportion is 0.171550013283
        Action: shift down
        New shift value 0.5
MSA 3 where current shifter value is 2.14
    Target proportion is 0.0842
    Simulated proportion is 0.0242370259247
        Action: shift up
        New shift value 2.15
MSA 4 where current shifter value is 1.73
    Target proportion is 0.16274
    Simulated proportion is 0.148234963199
        No action
MSA 5 where current shifter value is 1.63
    Target proportion is 0.141933
   